In [1]:
# coding: utf-8
from classifier import Classifier
from neo_models import Neo4j
from random import shuffle
from time import time

In [2]:
# 读取neo4j内容
db = Neo4j()
db.connectDB()
data_set = db.getLabeledHudongItem('labels.txt')

create neo4j class ...
load LabeledHudongItem over ...


In [3]:
# 读取fasttext的模型
shuffle(data_set)
length = int(len(data_set)/10)

vis = []  # 记录下每个label出现次数，控制每个label出现的个数
for i in range(17):
    vis.append(0)

cur = []  # 让每类样本尽量分布均匀
for p in data_set:
    label = p.label
    if vis[int(label)] > 30 and int(label) != 0:
        continue
    if vis[int(label)] > 50 and int(label) == 0:
        continue
    vis[int(label)] += 1
    cur.append(p)
    
data_set = cur
print(len(data_set))
# split = len(data_set)*199/200
# split = int(split) # 切分点
# print(split)
# train_set = data_set[:split]
# test_set = data_set[split:]
classifier = Classifier('wiki.zh.bin')

547
classifier load over...


In [4]:
def cross_verification(k_cv,data_set,W,k_num):  # k折叠交叉验证
    length = int(len(data_set)/k_cv)
    l = 0
    aver = 0.0
    for i in range(k_cv):
        train_set = data_set[:length*i]
        test_set = data_set[length*i:length*(i+1)]
        train_set.extend(data_set[length*(i+1):])
        classifier.load_trainSet(train_set)
        classifier.set_parameter(weight=W,k=k_num)
        count = 0
        for p in test_set:
            predict = classifier.KNN_predict(p)
            if predict==int(p.label):
                count += 1
            #print(str(p.title)+" "+str(predict)+"  "+str(p.label))
        precision = 1.0*count/len(test_set)
        aver += precision
      #  print("precision: "+str(1.0*count/len(test_set)*100)+"%")
    aver /= k_cv
   # print("aver precision:"+str(aver*100)+"%")
    return aver
    

In [5]:
import logging
logging.basicConfig(level=logging.DEBUG,
                format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                datefmt='%a, %d %b %Y %H:%M:%S',
                filename='mylog.log',
                filemode='w')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

grid_list = [0.1,0.3,1.,3.,10]
W = [0.,0.,0.,0.,0.]
ans = 0.
ans_W = [0.,0.,0.,0.,0.]
count = 0
k = 10  # knn的k

for w2 in grid_list:
    for w3 in grid_list:
        for w4 in grid_list:
            W = [1.0,w2,w3,w4,0.]
            cur = cross_verification(10,data_set,W,k)
            if cur > ans:
                ans = cur
                ans_W = W
                ans_k = k
            count += 1
            logging.info("count:"+str(count)+"precision: "+str(cur)+" now answer: "+" w="+str(ans_W))


root        : INFO     count:1precision: 0.6586956521739131 now answer:  w=[1.0, 0.1, 0.1, 0.1, 0.0]
root        : INFO     count:2precision: 0.6608695652173913 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:3precision: 0.6608695652173913 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:4precision: 0.6608695652173913 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:5precision: 0.6608695652173913 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:6precision: 0.641304347826087 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:7precision: 0.641304347826087 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:8precision: 0.6434782608695652 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:9precision: 0.6434782608695652 now answer:  w=[1.0, 0.1, 0.1, 0.3, 0.0]
root        : INFO     count:10precision: 0.6434782608695652 now answer:  w=[1.0, 0.1, 0.1, 0

root        : INFO     count:82precision: 0.641304347826087 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:83precision: 0.6434782608695653 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:84precision: 0.6456521739130434 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:85precision: 0.6456521739130434 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:86precision: 0.6130434782608696 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:87precision: 0.6260869565217391 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:88precision: 0.6260869565217391 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:89precision: 0.6304347826086956 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:90precision: 0.6304347826086956 now answer:  w=[1.0, 0.3, 0.1, 0.3, 0.0]
root        : INFO     count:91precision: 0.5869565217391305 now answer:  w=[1.0, 0

In [7]:
cur = cross_verification(10,data_set,[1.,2.,0.,2.,0],10)
ans = -7
ans_W = None
ans_k = None
if cur > ans:
    ans = cur
    ans_W = W
    ans_k = k
logging.info("count:"+str(count)+"precision: "+str(cur)+" now answer: "+" w="+str(ans_W))



KeyboardInterrupt: 

In [19]:
from hudong_class import HudongItem
p1 = HudongItem(db.matchHudongItembyTitle('旗月鱼'))
p2 = HudongItem(db.matchHudongItembyTitle('终霜'))
classifier.set_parameter(weight=[0.2,0.2,0.2,0.2,0.0],k=10)
classifier.KNN_predict(p2)

0.02350459626925365 0.0 0.0 0.0
锋面雨----0.04671318838685446  8
原浆----0.04605804840682626  9
单交种----0.04448714170742093  15
温室栽培----0.044089790501554586  15
玉米脱粒机----0.04399010501243235  14
雹害----0.04388583106996902  11
台风----0.043850393064465504  11
冷害----0.043680227090469294  11
霜冻----0.04352687991141527  11
霜冻----0.04352687991141527  11


11